In [1]:
import math 
from math import factorial
import numpy as np

In [40]:
def binomial_tree(spot, strike, delta_t, r, u, d, n_steps, call_put='call'):

    p = (math.exp(r*delta_t)-d)/(u-d)

    coefficients = []
    terms = {}
    for i in np.arange(n_steps+1):
        coefficient = factorial(n_steps)//(factorial(i)*(factorial(n_steps-i)))
        coefficients.append(coefficient)
        terms[i] = p**(n_steps-i)*((1-p)**(i)) 

    tree = {}
    for i in np.arange(n_steps+1):
        inner_tree = {}
        for j in np.arange(start=0, stop=i+1):
            inner_tree[j] = spot*(u**(i-j))*(d**(j))
        tree[i] = inner_tree
    
    fs = {}
    opt_tree = {}
    opt_tree_last = {}

    if call_put=='call':
        for k,v in tree[n_steps].items():
            poff = np.max([v-strike,0])
            fs[k] = [poff, coefficients[k], terms[k]]
            opt_tree_last[k] = poff

    elif call_put=='put':
        for k,v in tree[n_steps].items():
            poff = np.max([strike-v,0])
            fs[k] = [poff, coefficients[k], terms[k]]
            opt_tree_last[k] = poff

    opt_tree[n_steps] = opt_tree_last
    
    polinom = []
    for k in fs.keys():
        polinom.append(np.prod(fs[k]))
    f = math.exp(-n_steps*r*delta_t)*np.sum(polinom)

    for i in list(tree.keys())[::-1][:-1]:
        dict_opt = {}
        for n in np.arange(i):
            fu = opt_tree[i][n]
            fd = opt_tree[i][n+1]
            f = math.exp(-r*delta_t)*(p*fu+(1-p)*fd)
            dict_opt[n] = f
        opt_tree[i-1] = dict_opt

    binom_tree = {}
    for k,v in tree.items():
        level_dict = {}
        for k1, prices in v.items():
            option_price = opt_tree[k][k1]
            level_dict[k1] = [np.round(prices,4), np.round(option_price,4)]
        binom_tree[k] = level_dict

    return {'price':np.round(f,4), 'tree':binom_tree}

In [48]:
binomial_tree(spot=100, strike=100, delta_t=0.5, r=0.07, u=1.25, d=0.8, n_steps=3, call_put='put')

{'price': 11.1699,
 'tree': {0: {0: [100.0, 11.1699]},
  1: {0: [125.0, 4.2323], 1: [80.0, 19.6299]},
  2: {0: [156.25, 0.0], 1: [100.0, 9.2003], 2: [64.0, 32.5605]},
  3: {0: [195.3125, 0.0], 1: [125.0, 0.0], 2: [80.0, 20.0], 3: [51.2, 48.8]}}}